In [3]:
# !pip install fastembed

## Q1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`.
Use the `'jinaai/jina-embeddings-v2-small-en'` model. 

You should get a numpy array of size 512.

What's the minimal value in this array?


In [1]:
from fastembed.embedding import DefaultEmbedding
import numpy as np

# Load model
model = DefaultEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Embed query
query = "I just discovered the course. Can I join now?"
embedding = next(model.embed([query]))

# Check the shape and min value
print("Shape:", len(embedding))  # Should be 512
print("Min value:", min(embedding))


2025-07-12 08:40:31.858 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


Shape: 512
Min value: -0.11726373885183883


## A1.

The ninimal value is "-0.11".

## Q2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query
and the vector for the document?

In [2]:
query_vec = np.array(embedding)
doc_embedding = next(model.embed(["Can I still join the course after the start date?"]))
doc_vec = np.array(doc_embedding)

cos_sim = query_vec.dot(doc_vec)
print("Cosine similarity:", cos_sim)


Cosine similarity: 0.9008528895674548


## A2.

The cosine similarity is "0.9"

## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

In [3]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

Compute the embeddings for the text field, and compute the 
cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

In [4]:
doc_texts = [doc['text'] for doc in documents]
doc_embeddings = list(model.embed(doc_texts))

# Convert to matrix
V = np.array(doc_embeddings)
scores = V.dot(query_vec)

# Find highest score index
print("Best match index:", np.argmax(scores))


Best match index: 1


## A3.

The Doc index with highest similarity is "1".

## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of
`question` and `text`:

```python
full_text = doc['question'] + ' ' + doc['text']
``` 

Embed this field and compute the cosine between it and the
query vector. What's the highest scoring document?


In [5]:
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
full_embeddings = list(model.embed(full_texts))

V2 = np.array(full_embeddings)
scores2 = V2.dot(query_vec)

print("Best match index (Q4):", np.argmax(scores2))


Best match index (Q4): 0


## A4.

the highest scored doc is "0".

## Q5. Selecting the embedding model

Now let's select a smaller embedding model.
What's the smallest dimensionality for models in fastembed?


In [6]:
model2 = DefaultEmbedding(model_name="BAAI/bge-small-en")
embedding2 = next(model2.embed(["test"]))
print(len(embedding2))  # Shows the dimensionality


384


## A5.

The dim for the "BAAI/bge-small-en" model is 384.

## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

```python
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

```python
text = doc['question'] + ' ' + doc['text']
```

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results?
(The score for the first returned record):

In [7]:
# !pip install qdrant-client


In [4]:
from fastembed.embedding import DefaultEmbedding

model2 = DefaultEmbedding(model_name="BAAI/bge-small-en")


/Users/aidenerdogan/.pyenv/versions/3.11.12/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-12 10:52:10.616 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.
Fetching 5 files: 100%|██████████| 5/5 [00:04<00:00,  1.10it/s]


In [7]:
# !pip install qdrant-client

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, PointStruct
import uuid
import requests

# Get documents
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
documents_raw = requests.get(docs_url).json()

# Filter & prepare
documents = []
for course in documents_raw:
    if course['course'] != 'machine-learning-zoomcamp':
        continue
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

# Create texts to embed
texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
embeddings = list(model2.embed(texts))

# Connect to Qdrant
client = QdrantClient(host="localhost", port=6333) # for memory limits I have installed qdrant in docker on my local machine
client.recreate_collection(
    collection_name="ml_faq",
    vectors_config=VectorParams(size=len(embeddings[0]), distance=Distance.COSINE),
)

# Insert into Qdrant
points = [
    PointStruct(id=i, vector=embeddings[i], payload=documents[i])
    for i in range(len(embeddings))
]
client.upsert(collection_name="ml_faq", points=points)

# Query with Q1 question
query_vec = next(model2.embed(["I just discovered the course. Can I join now?"]))

search_result = client.search(collection_name="ml_faq", query_vector=query_vec, limit=1)
print("Top result score:", search_result[0].score)


/var/folders/h2/nzdrmmqs66x13b1h4f0882fh0000gn/T/ipykernel_91584/2922323109.py:25: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Top result score: 0.8703173


/var/folders/h2/nzdrmmqs66x13b1h4f0882fh0000gn/T/ipykernel_91584/2922323109.py:40: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(collection_name="ml_faq", query_vector=query_vec, limit=1)


## A6.

The highest result is "0.87"